In the previous article we covered **Batch Gradient Descent**. It trains the model feeding it with **ALL** the training examples **AT ONCE**. 

### Stochastic Gradient Descent
As you can imagine, for large datasets, this can be quite computationally intensive and time-consuming. This is where **Stochastic Gradient Descent** comes into play. Instead of using the entire dataset to calculate the gradient, SGD randomly selects just one training example to compute the gradient in each iteration and the parameters ($w_{1..n}$ and $b$) are updated **after each example**.

Think of this process as if you were again descending a mountain, but this time in thick fog with limited visibility. Rather than viewing the entire landscape to decide your next step, you make your decision based on where your foot lands next. This step is small and random, but it’s repeated many times, each time adjusting your path slightly in response to the immediate terrain under your feet.

#### Advantages
This stochastic nature of the algorithm provides several benefits:

* **Speed**: By using only a small subset of data at a time, SGD can make rapid progress in reducing the loss, especially for large datasets.

* **Escape from Local Minima**: The randomness helps SGD to potentially escape local minima, a common problem in complex optimization problems.

* **Online Learning**: SGD is well-suited for online learning, where the model needs to be updated as new data comes in, due to its ability to update the model incrementally.

#### Disadvantages
However, the stochastic nature also introduces variability in the path to convergence. The algorithm doesn’t smoothly descend towards the minimum; rather, it takes a more zigzag path, which can sometimes make the convergence process appear erratic.

### 1. Calculate Y
Again, we need to create a function that calculates a $y$ first.

$$ y = b + \vec{w} \cdot \vec{x} $$

where the size of $ \vec{w}, \vec{x} $ is all the same ($n$).

$$ n - \text{number of features} $$
$$ m - \text{number of observations}$$

In [141]:
import numpy as np

In [142]:
def calculate_y(x: list[float], w: list[float], b: float) -> list[float]:
    '''
    Calculate the predicted value for a single observation.

    Args:
        x -> list[float]    : list of features (x1, x2, ..., xn)
        w -> list[float]    : list of weights (w1, w2, ..., wn)
        b -> float          : bias

    Returns:
        y -> float          : predicted value for a single observation
    '''
    n = len(x) # Number of features
    y = b
    for k in range(n):
        y += w[k] * x[k] # y = w1*x1 + w2*x2 + ... + wn*xn + b
    return y

### 2. Calculate J (cost for single observation)
Now, we need to calculate the cost (the delta between predicted $\hat{y}$ and actual $y$). But just for a single observation and not for the whole set (i.e. no sum here).
$$ J^{(i)} = (\hat{y}^{(i)} - y^{(i)})^2 $$

In [143]:
def calculate_cost(y: float, x: list[float], w: list[float], b: float) -> float:
    '''
    Calculate the cost by comparing the predicted and actual value for SINGLE observation.

    Args:
        y -> float              : expected float value (y)
        x -> list[float]        : n features (x1, x2, ..., xn)
        w -> list[float]        : n weights (w1, w2, ..., wn)
        b -> float              : bias

    Returns:
        cost -> float           : cost for this observation
    '''
    y_pred = calculate_y(x, w, b) # Calculate the predicted value for the this observation
    return (y_pred - y) ** 2 # Calculate the squared difference between the predicted and actual value

### 3. Calculate dJ/dw and dJ/db (cost derivatives)
We need to calculate the derivative of $J^{(i)}$ - once w.r.t to $w$ and once w.r.t to $b$. This way we know how strongly a change in $w$ and $b$ impacts $J^{(i)}$.

$$ \frac{dJ^{(i)}}{dw_k} = 2 (\hat{y}^{(i)}  - y^{(i)}) x^{(i)}_k  $$
$$ \frac{dJ^{(i)}}{db} = 2 (\hat{y}^{(i)}  - y^{(i)}) $$

Where $i$ is the index of the particular observation we are using for executing the gradient descent and $k$ is the index of the feature. Note that for each feature ($n$ in total) we calculate a separate $\frac{dJ^{(i)}}{dw_k}$ derivative.

In [144]:
def calculate_dj_dw(y: float, x: list[float], w: list[float], b: float) -> list[float]:
    '''
    Calculate the derivative of the cost function with respect to each weight.

    Args:
        y -> float              : expected float value (y)
        x -> list[float]        : n features (x1, x2, ..., xn)
        w -> list[float]        : n weights (w1, w2, ..., wn)
        b -> float              : bias

    Returns:
        dj_dw -> list[float]    : n weight derivatives (dj_dw1, dj_dw2, ..., dj_dwn)
    '''
    n = len(w) # Number of features
    dj_dw = [0] * n # Initialize the derivative of the cost function with respect to the weights. n-sized list of zeros.

    y_pred = calculate_y(x, w, b) # Calculate the predicted value for the current observation
    for j in range(n): # For each feature
        dj_dw[j] += 2 * (y_pred - y) * x[j] # Calculate the derivative of the cost function with respect to the weights

    return dj_dw

def calculate_dj_db(y: float, x: list[float], w: list[float], b: float) -> float:
    '''
    Calculate the derivative of the cost function with respect to the bias.

    Args:
        y -> float              : expected float value (y)
        x -> list[float]        : n features (x1, x2, ..., xn)
        w -> list[float]        : n weights (w1, w2, ..., wn)
        b -> float              : bias

    Returns:
        dj_db -> float          : derivative of the cost function with respect to the bias
    '''
    y_pred = calculate_y(x, w, b) # Calculate the predicted value for the current observation
    dj_db = 2 * (y_pred - y) # Calculate the derivative of the cost function with respect to the bias
    return dj_db

### 4. Gradient Descent
Once we know how each weight affects the cost we can slightly update these weights in the direction we want (up or down), based on the derivative result (negative or positive). 

The term ‘stochastic’ refers to a system or process that is linked with a **random** probability. In each iteration of the training process, SGD randomly selects a single data point from the entire dataset. This randomness is what makes it **‘stochastic’**.

In [145]:
def gradient_descent(X: list[list[float]], Y: list[float], w: list[float], b: float, learn_rate: float, num_iterations: int) -> tuple[list[float], float]:
    '''
    Perform stochasticgradient descent to find the optimal weights and bias.

    Args:
        X -> list[list[float]]  : m rows of n features each (x^1, x^2, ..., x^m)
        Y -> list[float]        : m rows of expected float values (y^1, y^2, ..., y^m)
        w -> list[float]        : n weights (w1, w2, ..., wn)
        b -> float              : bias
        learn_rate -> float     : learning rate
        num_iterations -> int   : number of iterations to perform

    Returns:
        w -> list[float]        : n weights (w1, w2, ..., wn)
        b -> float              : bias
    '''
    for iteration_number in range(num_iterations):

        # Randomize the order of the observations
        m = len(X)
        random_indices = np.random.permutation(m)
        X_shuffled = X[random_indices]
        Y_shuffled = Y[random_indices]

        for i in range(m):
            x = X_shuffled[i] # Get a single observation
            y = Y_shuffled[i] # Get the expected value for that observation

            dj_dw = calculate_dj_dw(y, x, w, b) # Calculate the derivative of the cost function with respect to the weights
            dj_db = calculate_dj_db(y, x, w, b) # Calculate the derivative of the cost function with respect to the bias

            # Update the weights and bias
            w = [w[j] - learn_rate * dj_dw[j] for j in range(len(w))]
            b = b - learn_rate * dj_db

        # Print the cost every 100 iterations
        if iteration_number % 1000 == 0:
            print(f"Iteration {iteration_number}: Total cost = {calculate_total_cost(X, Y, w, b)}")
    return w, b

def calculate_total_cost(X: np.ndarray, Y: np.ndarray, w: list[float], b: float) -> float:
    '''Calculate the total cost by comparing the predicted and actual values for ALL observations.'''
    m = len(Y) # Number of observations
    cost = 0
    for i in range(m):                      # For each observation
        y_pred_i = X[i] @ w + b             # Calculate the predicted value for the current observation
        cost += (y_pred_i - Y[i]) ** 2      # Calculate the squared difference between the predicted and actual values
    return cost / m                         # Normalize by the number of observations

### 5. Demo
Let's do the well-known demo with **salary prediction**, where `salary` ($y$) is predicted by using `year of experience` ($x_1$) and `education level` ($x_2$).

In [146]:
# X1 = years of experience
X1 = [1.2, 1.3, 1.5, 1.8, 2, 2.1, 2.2, 2.5, 2.8, 2.9, 3.1, 3.3, 3.5, 3.8, 4, 4.1, 4.5, 4.9, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 10, 11, 12, 13, 14, 15]
# X2 = level of education 
X2 = [2, 5, 3, 5, 3, 4, 2, 3, 4, 4, 3, 7, 5, 6, 5, 5, 2, 3, 4, 5, 6, 7, 5, 3, 2, 4, 5, 7, 3, 5, 7, 7, 5]
# Y = salary
Y = [2900, 3300, 3100, 4200, 3500, 3800, 3300, 3500, 3750, 4000, 3900, 5300, 4420, 5000, 4900, 5200, 3900, 4800, 5700, 6500, 6930, 7500, 7360, 6970, 6800, 7500, 8000, 9500, 11000, 9500, 12300, 13700, 12500]

In [147]:
# Merge the X1 and X2 into a single X
X = [[x1, x2] for x1, x2 in zip(X1, X2)]

# Convert the X, Y to a numpy arrays
X = np.array(X)
Y = np.array(Y)

# Configure gradient descent settings
b_init = 0          # initial bias (could be random as well)
w_init = [0, 0]     # initial weights (could be random as well)
learn_rate = 0.00001   # learning rate (step size)
iterations = 100000   # number of iterations (epochs)

# Execute the gradient descent in search for optimal cost function (optimal `w` and `b`)
w, b = gradient_descent(X, Y, w_init, b_init, learn_rate, iterations)
print(f"Final `b` is {b} and `w` are {w}")

Iteration 0: Total cost = 43011296.25774596
Iteration 1000: Total cost = 318186.91040476674
Iteration 2000: Total cost = 309101.7593776509
Iteration 3000: Total cost = 301190.1557361399
Iteration 4000: Total cost = 294300.09965818824
Iteration 5000: Total cost = 288300.2367203786
Iteration 6000: Total cost = 283074.57090358174
Iteration 7000: Total cost = 278524.21185012016
Iteration 8000: Total cost = 274561.4072313068
Iteration 9000: Total cost = 271110.522816652
Iteration 10000: Total cost = 268105.09922716545
Iteration 11000: Total cost = 265488.051391522
Iteration 12000: Total cost = 263208.72562447545
Iteration 13000: Total cost = 261223.98199592295
Iteration 14000: Total cost = 259495.34535411824
Iteration 15000: Total cost = 257990.20651471458
Iteration 16000: Total cost = 256679.23133047236
Iteration 17000: Total cost = 255537.72665836426
Iteration 18000: Total cost = 254543.42066155915
Iteration 19000: Total cost = 253677.64594415313
Iteration 20000: Total cost = 252923.38491

### Conclusion
The final results in terms of cost and weights are basically the same as in the BGD. Yet, the stochastic gradient descent executes its training epochs quite faster. 

However, as you can see it requires **20x** more epochs of training in comparison with the BGD, so the speed factor is basically neutralized:
* `100 000` vs `5 000` epochs

Another observation is that in order to have stable loss minimization curve we need a significantly smaller learning rate here:
* `0.00001` vs `0.01` learning rate

### References
* [The Math Behind Stochastic Gradient Descent](https://towardsdatascience.com/stochastic-gradient-descent-math-and-python-code-35b5e66d6f79/)